In [ ]:
# %pip install numpy
# %pip install opencv_python_headless
# %pip install torch
# %pip install matplotlib
# %pip install pandas
# %pip install torchvision

In [1]:
# Importing Necessary libraries
import pandas as pd
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import LabelEncoder
import ast
import torch
%matplotlib inline


In [9]:
# Define a function to create the image dataset and labels
def create_dataset():
    img_data_array = [] 
    class_name = []      
    
    
    data = torch.load("crop_data.pt")
    
    for item in data:
        label = item["label"]
        image_tensor = item["image"]

        img_data_array.append(image_tensor)
        class_name.append(label)

    return img_data_array, class_name


In [ ]:
img_data , class_name = create_dataset()
class_name = np.array(class_name)
class_name = class_name.ravel()
le = LabelEncoder()
class_name = le.fit_transform(class_name)
len(img_data)
print(class_name)
len(class_name)

In [ ]:
print(f"First image shape: {img_data[0].shape}")
print(f"First label (age, weight): {class_name[0]}")

---

### Implementing a CNN in PyTorch

In [12]:
# Importing necessary libraries
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import torch.utils.data as Data
from torch import Tensor
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [ ]:
# version of pytorch
print(torch.__version__)

In [ ]:
images_tensor = img_data # Image data as tensor
labels_tensor = torch.tensor(class_name)  # Labels as tensor

train_images, test_images, train_labels, test_labels = train_test_split(
    images_tensor, labels_tensor, test_size=0.3, random_state=42)

torch_dataset_train = Data.TensorDataset(
    torch.Tensor(np.array(train_images)),  # Image data converted to a PyTorch tensor
    torch.Tensor(np.array(train_labels))  # Class labels converted to a PyTorch tensor
)

torch_dataset_test = Data.TensorDataset(
    torch.Tensor(np.array(test_images)),  # Image data converted to a PyTorch tensor
    torch.Tensor(np.array(test_labels))  # Class labels converted to a PyTorch tensor
)


In [15]:
# Define the trainloader and testloader for batching and shuffling data
trainloader = Data.DataLoader(
    torch_dataset_train,  # Training dataset
    batch_size=8,         # Batch size for training
    shuffle=True          # Shuffle the training data
)

testloader = Data.DataLoader(
    torch_dataset_test,   # Testing dataset
    batch_size=8,         # Batch size for testing
    shuffle=True          # Shuffle the testing data
)


In [ ]:
# Create an iterator for the training data loader
dataiter = iter(trainloader)

# Get the next batch of data
images = next(dataiter)

# Access the shape of the images in the batch
images[0].shape


In [ ]:
# Define the CNN model architecture
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()

        self.cnn_layers = nn.Sequential(
            # Define a 2D convolution layer with 3 input channels, 16 output channels, and a 5x5 kernel
            nn.Conv2d(3, 16, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Define another convolution layer with 16 input channels, 3 output channels, and a 50x50 kernel
            nn.Conv2d(16, 3, kernel_size=(50, 50), stride=(1, 1)),
            nn.MaxPool2d(kernel_size=1, stride=1, padding=0, ceil_mode=False)
        )

        self.linear_layers = nn.Sequential(
            # Define a fully connected layer with 3 input features and 3 output features
            nn.Linear(3, 21)
        )

    # Define the forward pass
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
# Define the CNN model architecture
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()

        self.cnn_layers = nn.Sequential(
            # Define a 2D convolution layer with 3 input channels, 16 output channels, and a 5x5 kernel
            nn.Conv2d(3, 16, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Define another convolution layer with 16 input channels, 3 output channels, and a 50x50 kernel
            nn.Conv2d(16, 3, kernel_size=(50, 50), stride=(1, 1)),
            nn.MaxPool2d(kernel_size=1, stride=1, padding=0, ceil_mode=False)
        )

        self.linear_layers = nn.Sequential(
            # Define a fully connected layer with 3 input features and 3 output features
            nn.Linear(3, 21)
        )

    # Define the forward pass
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x


# Define the model, optimizer, and loss function
model = CNNNet()  # Create an instance of the CNNNet class
optimizer = optim.SGD(model.parameters(), lr=0.0001)  # Define the optimizer with a learning rate of 0.0001
criterion = nn.CrossEntropyLoss()  # Define the loss function as Cross-Entropy Loss

# Check if GPU is available and move the model and loss function to the GPU if it is
print(torch.cuda.is_available())
if torch.cuda.is_available():
    model = model.to("cuda")
    criterion = criterion.to("cuda")

# Print the model architecture
print(model)


In [ ]:
# CUDA launch functions synchronous - the program will wait for the GPU kernel to complete before proceeding.
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
# Training the model for 30 epochs

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(10):
    running_loss = 0   # To accumulate training loss
    correct_train = 0  # To track correct predictions during training
    total_train = 0    # Total number of samples for training accuracy
    
    model.train()  # Set the model to training mode
    for images, labels in trainloader:
        if torch.cuda.is_available():
            images = images.to("cuda")
            labels = labels.to("cuda")

        # Reset the gradients to zero
        optimizer.zero_grad()

        # Forward pass
        output = model(images)
        labels = labels.type(torch.long)
        
        # Calculate the loss
        loss = criterion(output, labels)

        # Backpropagation
        loss.backward()

        # Update the model's weights
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(output, 1)  # Get predicted class
        correct_train += (predicted == labels).sum().item()  # Count correct predictions
        total_train += labels.size(0)  # Total number of samples in the batch

    # Calculate and print training accuracy
    train_accuracy = correct_train / total_train * 100
    avg_train_loss = running_loss / len(trainloader)

    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    correct_val = 0  # To track correct predictions during validation
    total_val = 0    # Total number of samples for validation accuracy
    val_loss = 0     # To accumulate validation loss
    
    with torch.no_grad():  # Disable gradient calculation for validation
        for val_images, val_labels in testloader:
            if torch.cuda.is_available():
                val_images = val_images.to("cuda")
                val_labels = val_labels.to("cuda")

            # Forward pass on validation set
            val_output = model(val_images)
            val_labels = val_labels.type(torch.long)
            # Calculate validation loss
            v_loss = criterion(val_output, val_labels)
            val_loss += v_loss.item()

            # Calculate validation accuracy
            _, val_predicted = torch.max(val_output, 1)
            correct_val += (val_predicted == val_labels).sum().item()
            total_val += val_labels.size(0)

    # Calculate validation accuracy
    val_accuracy = correct_val / total_val * 100
    avg_val_loss = val_loss / len(testloader)  # Average validation loss per batch

    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    # Print training and validation metrics for the epoch
    print(f"Epoch {epoch+1} - "
          f"Training loss: {avg_train_loss:.4f}, "
          f"Training accuracy: {train_accuracy:.4f}, "
          f"Validation loss: {avg_val_loss:.4f}, "
          f"Validation accuracy: {val_accuracy:.4f}")


In [ ]:
# Plot Loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

# Plot Accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_accuracies, label="Training Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.show()


In [21]:
# Define the file path where the model will be saved
filepath = 'cnn.pt'

# Save the model's state dictionary to the specified file path
torch.save(model.state_dict(), filepath)


In [22]:
# model_trained = CNNNet()
# model_trained.load_state_dict(torch.load(filepath))

In [23]:
# device = "cuda"--> use if you have GPU

In [ ]:
# [.2, .5, .3]
# y_pred_list = []
# y_true_list = []
# with torch.no_grad():
#     for x_batch, y_batch in testloader:
#         x_batch, y_batch = x_batch.to(device), y_batch.to(device)
#         y_test_pred = model(x_batch)
#         print(y_test_pred)
#         _, y_pred_tag = torch.max(y_test_pred, dim = 1)
#         y_pred_list.extend(y_pred_tag.cpu().numpy())
#         y_true_list.extend(y_batch.cpu().numpy())

###

# Prediction 
# Create empty lists to store predicted and true labels
y_pred_list = []  # Predicted labels
y_true_list = []  # True labels

# Disable gradient calculations for inference
with torch.no_grad():
    for x_batch, y_batch in testloader:
        x_batch, y_batch = x_batch.to(), y_batch.to()
        
        # Perform inference with the model
        y_test_pred = model(x_batch)
        print(y_test_pred)  # Display the model's predictions
        
        # Find the predicted class labels
        _, y_pred_tag = torch.max(y_test_pred, dim=1)
        
        # Extend the lists with the predicted and true labels
        y_pred_list.extend(y_pred_tag.cpu().numpy())
        y_true_list.extend(y_batch.cpu().numpy())


In [25]:
# Extract the maximum values (class indices) from the true labels
y_true_list_max = [m.argmax() for m in y_true_list]


In [ ]:
# Initialize variables to count correct and total predictions
correct_count, all_count = 0, 0

# Compare predicted labels (y_pred_list) with true labels (y_true_list_max)
for i in range(len(y_pred_list)):
    if y_pred_list[i] == y_true_list_max[i]:
        correct_count += 1
    all_count += 1

# Calculate and print the model accuracy
accuracy = correct_count / all_count
print("\nModel Accuracy =", accuracy)


---

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Step 1: Load and preprocess the image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((200, 200)),  # Resize to model input size
        transforms.ToTensor(),         # Convert to tensor
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize
    ])
    image = Image.open(image_path).convert('RGB')  # Ensure it's an RGB image
    image = transform(image)
    return image.unsqueeze(0)  # Add batch dimension [1, C, H, W]

# Step 2: Load your trained model
model_trained = CNNNet()
model_trained.load_state_dict(torch.load("model32.pt", weights_only=True))
model_trained.eval()
# Step 3: Predict the class
def predict(image_path, temp = 3.0):
    image = preprocess_image(image_path)  # Preprocess the image
    with torch.no_grad():  # Disable gradient computation
        logits = model_trained(image)  # Pass the image through the model
        print(logits)
        probabilities = torch.softmax(logits, dim=1) 
        predicted_class = torch.argmax(logits, dim=1).item()  # Get the class index
        confidence = probabilities[0, predicted_class].item()
    return predicted_class, confidence

# Example usage
image_path = "1.png"  # Replace with your image path
predicted_class, confidence = predict(image_path)
print(f"Predicted Class: {le.inverse_transform([predicted_class])}")
print(f"Confidence Score: {confidence}")
